In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Network

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.fc1 = nn.Linear(81 * 32 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 3)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
#         print(x.shape)
        x = self.pool(nn.functional.relu(self.conv2(x)))
#         print(x.shape)
        x = x.view(-1, 81 * 32 * 6 * 6)
#         print(x.shape)
        x = nn.functional.relu(self.fc1(x))
#         print(x.shape)
        x = self.fc2(x)        
#         print(x.shape)
        return x
    
    
    
    
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(3, 16, 3)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(16, 32, 3)
#         self.fc1 = nn.Linear(32 * 6 * 6, 120)
#         self.fc2 = nn.Linear(120, 3)

#     def forward(self, x):
#         x = self.pool(nn.functional.relu(self.conv1(x)))
#         print(x.shape)
#         x = self.pool(nn.functional.relu(self.conv2(x)))
#         print(x.shape)
#         x = x.view(-1, 32 * 6 * 6)
#         print(x.shape)
#         x = nn.functional.relu(self.fc1(x))
#         print(x.shape)
#         x = self.fc2(x)        
#         print(x.shape)
#         return x

Data loading



In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import cv2 
import os
import numpy as np


def loadTrainData(train=False):
    results=[]
    numOfTrain = 80
    numOfTest = 20

    if train:
        # Train
        root_dir = "C:/Users/asus/Desktop/clf/train/Bed/"      

        for index in range(numOfTrain):
            im = Image.open(root_dir+f"1 ({index+1}).jpg")
            results.append({'x': im, 'y': torch.tensor(0)})
            
        root_dir = "C:/Users/asus/Desktop/clf/train/Chair/"
        for index in range(numOfTrain):
            im = Image.open(root_dir+f"1 ({index+1}).jpg")
            results.append({'x': im, 'y': torch.tensor(1)})
        
        root_dir = "C:/Users/asus/Desktop/clf/train/Sofa/"
        for index in range(numOfTrain):
            im = Image.open(root_dir+f"1 ({index+1}).jpg")
            results.append({'x': im, 'y': torch.tensor(2)})
        
    else:
        # Test
        root_dir = "C:/Users/asus/Desktop/clf/test/Bed/"
        for index in range(numOfTest):
            im = Image.open(root_dir+f"1 ({index+1}).jpg")
            results.append({'x': im, 'y': torch.tensor(0)})
            
        root_dir = "C:/Users/asus/Desktop/clf/test/Chair/"
        for index in range(numOfTest):
            im = Image.open(root_dir+f"1 ({index+1}).jpg")
            results.append({'x': im, 'y': torch.tensor(1)})
        
        root_dir = "C:/Users/asus/Desktop/clf/test/Sofa/"
        for index in range(numOfTest):
            im = Image.open(root_dir+f"1 ({index+1}).jpg")
            results.append({'x': im, 'y': torch.tensor(2)})

    print(len(results))
    return results
    



class CustomDataset(Dataset):
    def __init__(self, train=True, transform=None):

        self.samples = loadTrainData(train=train)
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        return self.transform(self.samples[index]['x']), self.samples[index]['y']



train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
train_dataset = CustomDataset(train=True, transform=train_transforms)
test_dataset = CustomDataset(train=False, transform=train_transforms)

240
60


In [4]:
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

Function loss and optimizer

In [5]:
net = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

Training and evaluation

In [6]:
num_epochs = 50
for epoch in range(num_epochs):
    running_loss = 0.0
    for (inputs, labels) in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)    
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()



In [7]:
modelPath = "./model"        
torch.save(net, modelPath)
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = correct / total
print(f'Epoch {epoch + 1} - Training Loss: {running_loss / len(train_loader)}, Test Accuracy: {test_accuracy:.2f}')

Epoch 50 - Training Loss: 0.5270397620896499, Test Accuracy: 0.78
